In [1]:
import re
import os
import pandas as pd
import numpy as np
import traceback2

where_map = {}
# pairs regexp: "\[([a-zA-z0-9$\-+τ\.%]*)\]-", "-\[([a-zA-z0-9$\-+τ]*)\]"

In [2]:
sm = {
    "WBTC" : "BTC",
    "WETH" : "ETH",
    "WBNB" : "BNB",
    "WFTM" : "FTM",
    "WAVAX" : "AVAX",
}


In [3]:
def pair1(line):
    try:
        is_3_pairs = line.split("]-[")
        if len(is_3_pairs) > 2:
            return None
        s_index = line.index("[") + 1
        s = line[s_index: line.index("]", s_index)]
        if all(ord(c) < 128 for c in s):
            return s
        else:
            print(line)
            return None
    except:
        return None

def pair2(line):
    s_index = line.index("-[") + 3
    s = line[s_index: line.index("]", s_index)]
    if all(ord(c) < 128 for c in s):
        return s
    else:
        print(line)
        return None

In [4]:
def apr(line):
    apr = "Year [0-9\,\.]+\%"
    x = re.findall(apr, line)
    return x[0].replace("Year ", "").replace("%", "").strip() if len(x) else 0

In [5]:
def tvl(line):
    tvl = "TVL: \$[0-9\.\,]+"
    x = re.findall(tvl, line)
    return x[0].replace("TVL: $", "").replace("%", "").replace(",","").strip()

In [6]:
def where(line):
    if 'INFO' in line:
        return line.replace("INFO", "").replace('\n', '').replace(":", "").replace("//", "://").strip()
    return None

In [7]:
def get_single_pair(line):
    exp = "\d* *.*Price:"
    x = re.findall(exp, line)
    if not len(x):
#         print(">>>>>>>>>>>",line)
        return
    pair = x[0].replace(" Price:", "")
    return pair[pair.rindex(" "):].strip() if " " in pair else pair

def get_single_tvl(line):
    exp = "\([0-9\,\.$]+\)"
    x = re.findall(exp, line)
    return x[0].replace("(", "").replace(")", "").replace("$", "").replace(",","").strip()


In [8]:
def read_blocks_from_files(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    for line in lines:
        line = line.strip()
        if line == "" or len(re.findall("INFO[ ]+: ", line)):
            blocks.append(block)
            if len(line):
                blocks.append([line])
            block = []
        else:
            block.append(line)
    blocks.append(block)
    return blocks

def get_pair_info_from_block(block, wh, network):
    if len(block) == 1:
        if 'INFO' in block[0]:
            return [where(block[0])]
        else:
            return [wh]
    if len(block) and pair1(block[0]):
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [pair1(block[0]), pair2(block[0]), tvl(block[0]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    elif len(block) > 2 and get_single_pair(block[0]):
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [get_single_pair(block[0]), "", get_single_tvl(block[1]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
        
    return None


def get_pairs(blocks, network):
    where = ""
    pairs = []
    for block in blocks:
        try:
            pair = get_pair_info_from_block(block, where, network)
            if len(where) == 0 and pair and len(pair) > 2:
                print(block, network)
            if pair:
                if len(pair) == 1 and pair[0] and len(pair[0]):
                    where = pair[0]
                else:
                    pairs.append(pair)
        except Exception:
            print(block[0])
    return pairs
        

In [42]:
data_dir = 'vfat_tool_data/2021-06-07_set/'
ignorable_files = ['.ipynb_checkpoints', "Yield Farm Manual 6:7 New .xlsx"]
def conver_files_to_one_df(data_dir):
    all_pairs = []
    files = os.listdir(data_dir)
    for file_name in files:
        if file_name in ignorable_files:
            continue
        network = file_name[0:file_name.index(" ")]
        if 'Alternate' != network:
            continue
        blocks = read_blocks_from_files(f"{data_dir}{file_name}")
        pairs = get_pairs(blocks, network)
        all_pairs.extend(pairs)
    return all_pairs

In [43]:
pairs = conver_files_to_one_df(data_dir)

[🐟]-[WETH] Uni LP [+] [-] [<=>] Price: $502.31 TVL: $179,862.69
[🐟]-[WETH] Uni LP [+] [-] [<=>] Price: $502.31 TVL: $179,862.69
[🐟]-[USDC] Uni LP [+] [-] [<=>] Price: $152,963,650.25 TVL: $49,697.90
[🐟]-[USDC] Uni LP [+] [-] [<=>] Price: $152,963,650.25 TVL: $49,697.90
[⛩️]-[🐟] Uni LP [+] [-] [<=>] Price: $5.77 TVL: $40,624.01
[⛩️]-[🐟] Uni LP [+] [-] [<=>] Price: $5.77 TVL: $40,624.01


In [44]:
pair_df = pd.DataFrame(pairs, columns=['pair1', 'pair2', 'tvl', 'apr', 'where', 'network'])
pair_df['apy'] = np.nan

In [45]:
pairs

[['UNI',
  '',
  '63387.82',
  '129.84',
  'https://penguinswap.eth.link',
  'Alternate'],
 ['BTCx5-USDC-1Jun21-LP',
  '',
  '1081817.32',
  '382.02',
  'https://app.compli.fi',
  'Alternate'],
 ['BTCx5-USDC-1Jun21-UP',
  '',
  '28785.49',
  '7178.55',
  'https://app.compli.fi',
  'Alternate'],
 ['BTCx5-USDC-1Jun21-DOWN',
  '',
  '55735.03',
  '3707.51',
  'https://app.compli.fi',
  'Alternate'],
 ['ETHx5-USDC-1Jun21-LP',
  '',
  '195994.73',
  '2108.61',
  'https://app.compli.fi',
  'Alternate'],
 ['ETHx5-USDC-1Jun21-UP',
  '',
  '250968.95',
  '823.36',
  'https://app.compli.fi',
  'Alternate'],
 ['ETHx5-USDC-1Jun21-DOWN',
  '',
  '282262.74',
  '732.08',
  'https://app.compli.fi',
  'Alternate'],
 ['LINKx5-USDC-1Jun21-LP',
  '',
  '49187.59',
  '8402.05',
  'https://app.compli.fi',
  'Alternate'],
 ['LINKx5-USDC-1Jun21-UP',
  '',
  '3824.53',
  '54029.73',
  'https://app.compli.fi',
  'Alternate'],
 ['LINKx5-USDC-1Jun21-DOWN',
  '',
  '0.00',
  0,
  'https://app.compli.fi',
  'Alter

In [13]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy
0,WFTM,CE,3425040.24,219.04,https://popsicle.finance,FATOM,NaN
1,WFTM,CE,3032060.91,239.93,https://popsicle.finance,FATOM,NaN
2,ICE,,155258.78,0.00,https://popsicle.finance,FATOM,NaN
3,WFTM,CE,3425040.24,0.00,https://popsicle.finance,FATOM,NaN
4,fUSDT+DAI+USDC,,27704.13,0.00,https://popsicle.finance,FATOM,NaN
...,...,...,...,...,...,...,...
2563,FUSE,SDC,78652.58,279.02,https://fuse.io,ETH,NaN
2564,FUSE,ETH,1025341.98,129.89,https://fuse.io,ETH,NaN
2565,WILD,ETH,895056.11,1700.00,https://wild.credit,ETH,NaN
2566,SPELL,ETH,12514487.48,195.06,https://abracadabra.money,ETH,NaN


In [14]:
pair_df[(pair_df['pair1'] == 'uUNICLY') | (pair_df['pair2'] == 'uARTBLOCKS')]

,pair1,pair2,tvl,apr,where,network,apy
2551,uUNICLY,ETH,353586.53,689.89,https://www.app.unic.ly/#/farm,ETH,NaN


In [15]:
sheet_pairs_df = pd.read_csv('MATIC 6_1 New.csv')
eth_sheet_pairs_df = pd.read_csv('eth_net_pairs.csv')
eth_sheet_pairs_df.columns=['pair1', 'pair2', 'tvl', 'where', 'apr']
eth_sheet_pairs_df['apy'] = np.nan
eth_sheet_pairs_df['network'] = 'ETH'
sheet_pairs_df = sheet_pairs_df.append(eth_sheet_pairs_df)

In [16]:
# sheet_pairs_df

In [17]:
def update_apr(apr):
    apr = f"{apr}".replace("%", "")
    if len(apr) == 0 or "nan" == apr:
        return np.nan
    else:
        return apr

In [18]:
pair_df = pair_df.append(sheet_pairs_df).reset_index(drop=True)
pair_df['apr'] = pair_df['apr'].apply(lambda x: update_apr(x))
pair_df['pair1'] = pair_df['pair1'].apply(lambda x: sm[x.strip()] if x.strip() in sm else x.strip())
pair_df['pair2'] = pair_df['pair2'].apply(lambda x: sm[x.strip()] if x.strip() in sm else x.strip())

In [19]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy
0,FTM,CE,3425040.24,219.04,https://popsicle.finance,FATOM,NaN
1,FTM,CE,3032060.91,239.93,https://popsicle.finance,FATOM,NaN
2,ICE,,155258.78,0.00,https://popsicle.finance,FATOM,NaN
3,FTM,CE,3425040.24,0.00,https://popsicle.finance,FATOM,NaN
4,fUSDT+DAI+USDC,,27704.13,0.00,https://popsicle.finance,FATOM,NaN
...,...,...,...,...,...,...,...
2967,UMB,ETH,"92,513.56",120.61,SUSHI,ETH,NaN
2968,MOVE,ETH,"763,577.19",29.23,SUSHI,ETH,NaN
2969,LEV,ETH,"235,399.41",18.96,SUSHI,ETH,NaN
2970,ETH,MLN,"3,445,685.39",64.77,SUSHI,ETH,NaN


In [20]:
# pair_df['apr'].unique().tolist()

In [21]:
def convert_apr_to_apy(pair_df):
    tmp_df = pair_df[pair_df['apy'].isna()]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apr = row['apr']
        if apr:
            apr = float(apr.replace(',',''))
        else:
            apr = 0
        try:
            pair_df.loc[index, 'apy'] = ((1+(apr/100)/365)**365-1)*100
        except:
            pair_df.loc[index, 'apy'] = np.nan
    return
            
def convert_apy_to_apr(pair_df):
    tmp_df = pair_df[(pair_df['apr'].isnull())]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apy = row['apy']
        if apy:
            apy = float(apy.replace(',',''))
        else:
            apy = 0
        try:
            pair_df.loc[index, 'apr'] = (((1+(apy/100))**(1/365)-1)*365)*100
        except:
            pair_df.loc[index, 'apr'] = np.nan
    return


convert_apr_to_apy(pair_df)
convert_apy_to_apr(pair_df)

(2795, 7)
(98, 7)


In [22]:
pair_df[pair_df['apy'].isna()]

,pair1,pair2,tvl,apr,where,network,apy
74,CREAM,,0.00,26607271.15,https://ftm.borgswap.exchange,FATOM,NaN
1120,MOONTREE,BEAR,35.21,60083983518.52,https://www.ybearswap.finance,BSC,NaN
2405,USDT,,7279.01,602183.35,https://www.bigdataprotocol.com/datavault,ETH,NaN
2406,USDC,,1022.71,4285970.71,https://www.bigdataprotocol.com/datavault,ETH,NaN
2422,USDC,,49.89,4006328.11,https://app.sirenmarkets.com/stake,ETH,NaN


In [39]:
pair_df[pair_df['apr'].isna()]

,pair1,pair2,tvl,apr,network,apy,dex


In [24]:
single_piar_df = pair_df[pair_df['pair2'] == ""].copy()
double_coin_pair_df = pair_df[pair_df['pair2'] != ""].copy()
tmp_df = double_coin_pair_df.copy()

In [25]:
single_piar_df

,pair1,pair2,tvl,apr,where,network,apy
2,ICE,,155258.78,0.00,https://popsicle.finance,FATOM,0.0
4,fUSDT+DAI+USDC,,27704.13,0.00,https://popsicle.finance,FATOM,0.0
26,BOO,,6500141.14,173.58,https://spookyswap.finance,FATOM,465.017004
27,SPIRIT,,5042230.23,78.69,https://app.spiritswap.finance,FATOM,119.471672
62,SON,,17306.81,195.10,https://ftm.borgswap.exchange,FATOM,599.925881
...,...,...,...,...,...,...,...
2540,LUSD3CRV-f,,116096081.26,0,https://www.convexfinance.com,ETH,0.0
2541,BUSD3CRV-f,,107458575.00,0,https://www.convexfinance.com,ETH,0.0
2542,alUSD3CRV-f,,259099840.38,0,https://www.convexfinance.com,ETH,0.0
2562,LEV,,139011.07,253.14,https://lever.network,ETH,1146.172956


In [26]:
double_coin_pair_df

,pair1,pair2,tvl,apr,where,network,apy
0,FTM,CE,3425040.24,219.04,https://popsicle.finance,FATOM,788.046372
1,FTM,CE,3032060.91,239.93,https://popsicle.finance,FATOM,992.931446
3,FTM,CE,3425040.24,0.00,https://popsicle.finance,FATOM,0.0
5,FTM,OO,10163940.14,200.84,https://spookyswap.finance,FATOM,641.047552
6,fUSDT,FTM,3371622.58,95.02,https://spookyswap.finance,FATOM,158.303566
...,...,...,...,...,...,...,...
2967,UMB,ETH,"92,513.56",120.61,SUSHI,ETH,233.379625
2968,MOVE,ETH,"763,577.19",29.23,SUSHI,ETH,33.934813
2969,LEV,ETH,"235,399.41",18.96,SUSHI,ETH,20.870649
2970,ETH,MLN,"3,445,685.39",64.77,SUSHI,ETH,91.004347


In [27]:
tmp_df

,pair1,pair2,tvl,apr,where,network,apy
0,FTM,CE,3425040.24,219.04,https://popsicle.finance,FATOM,788.046372
1,FTM,CE,3032060.91,239.93,https://popsicle.finance,FATOM,992.931446
3,FTM,CE,3425040.24,0.00,https://popsicle.finance,FATOM,0.0
5,FTM,OO,10163940.14,200.84,https://spookyswap.finance,FATOM,641.047552
6,fUSDT,FTM,3371622.58,95.02,https://spookyswap.finance,FATOM,158.303566
...,...,...,...,...,...,...,...
2967,UMB,ETH,"92,513.56",120.61,SUSHI,ETH,233.379625
2968,MOVE,ETH,"763,577.19",29.23,SUSHI,ETH,33.934813
2969,LEV,ETH,"235,399.41",18.96,SUSHI,ETH,20.870649
2970,ETH,MLN,"3,445,685.39",64.77,SUSHI,ETH,91.004347


In [28]:
tmp_df['tmp'] = tmp_df['pair1']
tmp_df['pair1'] = tmp_df['pair2']
tmp_df['pair2'] = tmp_df['tmp']
del tmp_df['tmp']

In [29]:
double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

,pair1,pair2,tvl,apr,where,network,apy
0,$DG,ETH,738323.48,95.76,https://quickswap.exchange/#/quick,MATIC,160.217083
1,$DG,QUICK,316608,93.335541,Quickswap,MATIC,154
2,$DG,UICK,308739.68,84.77,https://quickswap.exchange/#/quick,MATIC,133.197872
3,1INCH,BNB,1989.49,345.67,https://icecreamswap.finance,BSC,3020.050963
4,1INCH,BNB,12710.23,291.04,http://mangofarm.finance,BSC,1715.340213
...,...,...,...,...,...,...,...
4323,ycVT,umcha,13424.62,746.51,https://yumcha.finance,BSC,161835.925603
4324,yumcha,USD,58290.00,863.91,https://yumcha.finance,BSC,510640.49136
4325,yvBOOST,ETH,"22,005,820.00",50.71,SUSHI,ETH,65.988457
4326,zLOT,ETH,"54,701.00",40.80,SUSHI,ETH,50.346454


In [30]:
pair_df = double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)
pair_df = single_piar_df.append(pair_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

In [31]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy
0,$DG,ETH,738323.48,95.76,https://quickswap.exchange/#/quick,MATIC,160.217083
1,$DG,QUICK,316608,93.335541,Quickswap,MATIC,154
2,$DG,UICK,308739.68,84.77,https://quickswap.exchange/#/quick,MATIC,133.197872
3,1INCH,,0.00,0,https://caramelswap.finance,BSC,0.0
4,1INCH,BNB,1989.49,345.67,https://icecreamswap.finance,BSC,3020.050963
...,...,...,...,...,...,...,...
5131,yumcha,,0.00,0,https://yumcha.finance,BSC,0.0
5132,yumcha,USD,58290.00,863.91,https://yumcha.finance,BSC,510640.49136
5133,yvBOOST,ETH,"22,005,820.00",50.71,SUSHI,ETH,65.988457
5134,zLOT,ETH,"54,701.00",40.80,SUSHI,ETH,50.346454


In [32]:
pair_df = pair_df.drop_duplicates()

In [33]:
pair_df['dex'] = pair_df['where']
del pair_df['where']

<ipython-input-33-2bf09cd45164>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pair_df['dex'] = pair_df['where']


In [34]:
pair_df.to_csv('vfat_tools_data.csv')

In [35]:
pair_df['dex'].unique().tolist()

['https://quickswap.exchange/#/quick',
 'Quickswap',
 'https://caramelswap.finance',
 'https://icecreamswap.finance',
 'http://mangofarm.finance',
 'https://absorber.finance',
 'https://www.convexfinance.com',
 'https://ellipsis.finance',
 'https://frozenyogurt.finance',
 'https://beta.belt.fi',
 'https://blue.planetfinance.io',
 'https://polygaj.finance',
 'https://www.polycake.finance',
 'https://polylion.exchange',
 'https://www.moonwolf.io',
 'https://polygold.finance',
 'https://polypingu.finance',
 'https://polygon.prismfinance.net',
 'https://www.bigdataprotocol.com/datavault',
 'https://app.pangolin.exchange/#/png',
 'PolyCat',
 'Dfyn',
 'SUSHI',
 'Sushi',
 'https://app.elk.finance',
 'PolyZap',
 'Polylion',
 'PolyCake',
 'PolyGold',
 'https://www.knightsdefi.com',
 'hhttps://taco.finance/',
 'https://swipe.org/farm',
 'https://koaladefi.finance/',
 'https://app.hyruleswap.com',
 'https://limeswap.io',
 'https://vanillacake.farm',
 'https://garudaswap.finance',
 'https://gocerb

In [36]:
pair_df[pair_df['pair1'] == 'ETH']

,pair1,pair2,tvl,apr,network,apy,dex
1886,ETH,,1065.21,108.07,FATOM,194.204015,https://ftm.borgswap.exchange
1887,ETH,,175.06,556.41,FATOM,24916.388851,https://thegreenhouse.finance
1888,ETH,,6.37,77847.71,BSC,1037842848943826568478985412663639458915456380...,https://wantanmee.finance
1889,ETH,,520768.24,28.23,BSC,32.603182,https://koaladefi.finance/
1890,ETH,,167226.10,46.56,BSC,59.24971,https://app.hyruleswap.com
...,...,...,...,...,...,...,...
2327,ETH,xKNCa,7245.58,0.00,ETH,0.0,https://xtoken.cafe/app/dashboard
2328,ETH,xKNCb,7218.95,0.00,ETH,0.0,https://xtoken.cafe/app/dashboard
2329,ETH,xSUSHI,"40,446,441.00",19.31,ETH,21.294215,SUSHI
2330,ETH,yvBOOST,"22,005,820.00",50.71,ETH,65.988457,SUSHI


In [37]:
pair_df['apr'].unique().tolist()

['95.76',
 93.33554098784225,
 '84.77',
 '0',
 '345.67',
 '291.04',
 '0.00',
 '29.46',
 '17.91',
 '57.79',
 '76.66',
 '85.81',
 '568.48',
 '340.14',
 '560.05',
 '166350.20',
 '30.54',
 '43.8',
 '92',
 '12.95',
 33.92691860569297,
 '32.85',
 '104.34',
 '209.36',
 '85.01',
 '112.78',
 '1,335.95',
 '200.06',
 '392.75',
 '517.04',
 '160.27',
 '562.23',
 '518.53',
 '0.31',
 '37.94',
 '36.49',
 '26.31',
 '178.68',
 '155.71',
 '29.12',
 '106.17',
 '343.82',
 '52.13',
 '22.36',
 '184.18',
 '112.54',
 '3.48',
 '80.31',
 '332.97',
 '117.66',
 '199.03',
 '105.83',
 '100.57',
 '231.19',
 '553.52',
 '46.18',
 '173.50',
 '79.55',
 '49.04',
 '507.18',
 '153.72',
 '265.80',
 '309.88',
 '261.58',
 '167.15',
 '70.76',
 79.83790879433461,
 108.34099093708815,
 211.02107301560204,
 '90.88',
 '115.67',
 '61.22',
 '76.65',
 '40.24',
 '232.04',
 97.58618487063264,
 '61.02',
 '245.98',
 97.20710373810203,
 '92.24',
 '98.12',
 '111.21',
 '30.68',
 '287.91',
 '627.58',
 '99.27',
 '537.96',
 '56.02',
 '86.91',
 

In [38]:
pair_df

,pair1,pair2,tvl,apr,network,apy,dex
0,$DG,ETH,738323.48,95.76,MATIC,160.217083,https://quickswap.exchange/#/quick
1,$DG,QUICK,316608,93.335541,MATIC,154,Quickswap
2,$DG,UICK,308739.68,84.77,MATIC,133.197872,https://quickswap.exchange/#/quick
3,1INCH,,0.00,0,BSC,0.0,https://caramelswap.finance
4,1INCH,BNB,1989.49,345.67,BSC,3020.050963,https://icecreamswap.finance
...,...,...,...,...,...,...,...
5131,yumcha,,0.00,0,BSC,0.0,https://yumcha.finance
5132,yumcha,USD,58290.00,863.91,BSC,510640.49136,https://yumcha.finance
5133,yvBOOST,ETH,"22,005,820.00",50.71,ETH,65.988457,SUSHI
5134,zLOT,ETH,"54,701.00",40.80,ETH,50.346454,SUSHI
